In [1]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
import re
import numpy as np
import statistics
import math

In [2]:
recipes_in = pd.read_parquet('../01_DataCleansing/recipes_in.parquet')
users_in = pd.read_csv('../01_DataCleansing/users_in.csv')

### Only Keep Recipes that Have Values in Cuisine Column in This Test

In [49]:
recipes=recipes_in[recipes_in['cuisine']!="NO CUISINE"]

### Set Test Users As Those Who Rate At Least 500 Recipes as Non Zero

In [50]:
users_in_exclude0=users_in.copy()
users_in_exclude0=users_in_exclude0[users_in_exclude0['ratings']!=0]
users=list(set(users_in_exclude0['user_id']))
len(users)


24961

In [51]:
user_recipe_count=users_in_exclude0.groupby(['user_id']).count()
test_users=user_recipe_count[user_recipe_count['recipe_id']>=500].reset_index()
test_users=list(test_users['user_id'])
len(test_users)

193

In [52]:
yes=0
yes_low=0
for u in test_users:
    input_user_id=u
    idx = np.where((users_in['user_id']==input_user_id) & (users_in['ratings']>=4))
    live_user_recipes=users_in.copy()
    live_user_recipes=live_user_recipes.loc[idx][['user_id','recipe_id']]
    
    #Calculate similarity scores for all recipes given this user's recipe preferences
    cuisines = recipes['cuisine'].unique() # All unique cuisines
    user_cuisines = pd.DataFrame(live_user_recipes.merge(recipes,on='recipe_id')['cuisine'].value_counts()) # Value counts of cuisines for recipes user has rated
    user_pref=user_cuisines.copy()
    all_cuisines = pd.DataFrame(cuisines).set_index(0) # Creates dataframe with zero counts for all cuisines
    all_cuisines['cuisine'] = 0
    user_cuisines = pd.concat([user_cuisines,all_cuisines]) # Combines user cuisine counts and zero count dataframe
    user_cuisines = user_cuisines[~user_cuisines.index.duplicated(keep='first')] # Remove duplicate (zero) cuisines
    user_cuisines = (user_cuisines-user_cuisines.min())/(user_cuisines.max()-user_cuisines.min()) # Scale between 0 and 1
    cuisine_sim = ( # Merges cuisine similarity score with all recipes
           recipes
           .reset_index()
           .merge(user_cuisines,left_on='cuisine',right_on=user_cuisines.index)
           .set_index('recipe_id')[['cuisine_y']]
           .rename(columns={'cuisine_y':'cuisine_sim'})
       )
    cuisine_sim=cuisine_sim.reset_index()
    cuisine_rank=cuisine_sim.sort_values(by='cuisine_sim',ascending=False)
    #Get top 1 cuisine from user's top-rated recipes
    user_pref=user_pref.reset_index()
    user_pref=user_pref.rename(columns={"index": "Cuisine", "cuisine": "Count"})
    user_pref=user_pref.sort_values(by='Count',ascending=False)
    
    #Get top 1 cuisine from top 20 recipes from the similarity ranking
    high_rank=cuisine_rank[0:20]
    high_rank=pd.merge(high_rank,recipes,how="inner",on='recipe_id')
    import collections
    high=dict(collections.Counter(high_rank['cuisine']))
    high=sorted(high.items(), key=lambda kv: kv[1],reverse=True)
    
    #If top 1 cuisine from user's top-rated recipes equals top 1 cuisine from top 20 recipes from the similarity ranking, add 1 to "yes"
    if user_pref.iloc[0]['Cuisine']==high[0][0]:
        yes+=1
    
    #Get top 1 cuisine from bottom 20 recipes from the similarity ranking    
    low_rank=cuisine_rank[-21:-1]
    low_rank=pd.merge(low_rank,recipes,how="inner",on='recipe_id')
    low=dict(collections.Counter(low_rank['cuisine']))
    low=sorted(low.items(), key=lambda kv: kv[1],reverse=True)
    
    #If top 1 cuisine from user's top-rated recipes equals top 1 cuisine from bottom 20 recipes from the similarity ranking, add 1 to "yes_low"
    if user_pref.iloc[0]['Cuisine']==low[0][0]:
        yes_low+=1
    

In [53]:
yes

193

In [54]:
yes_low

0

193 out of the 193 test users have top 1 cuisine from user's top-rated recipes equals top 1 cuisine from top 20 recipes from the similarity ranking;

0 out of out of the 193 test users have top 1 cuisine from user's top-rated recipes equals top 1 cuisine from bottom 20 recipes from the similarity ranking